<a href="https://colab.research.google.com/github/yashk468/step-detection-/blob/main/Himu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import joblib
from scipy.signal import find_peaks # Included even if not currently used, for future expansion

# --- Global Definitions from your script ---
window_size = 20
step_size = 10
# Sampling rate was 150ms (0.15s). Convert to Hz (samples per second)
sampling_rate = 1 / 0.15
step_length_m = 0.70
weight_kg = 75 # Default weight
METs = {
    'flat': 3.5,     # Moderate walking
    'incline': 6.0,  # Brisk uphill walking
    'running': 15.0  # Running
}

# --- Load the Model ---
try:
    # Load the pre-trained RandomForestClassifier model
    model = joblib.load('incline_step_classifier.pkl')
except FileNotFoundError:
    st.error("Error: Model file 'incline_step_classifier.pkl' not found. Please ensure it is in the same directory.")
    model = None

# --- Feature Extraction Function (Adapted from your script) ---
def extract_features_advanced(df, window_size=window_size, step_size=step_size):
    """Extracts features from the time series data using a sliding window."""
    X = []

    # Calculate magnitude features once for efficiency
    if 'AccMag' not in df.columns:
        df['AccMag'] = np.sqrt(df['AccX']**2 + df['AccY']**2 + df['AccZ']**2)
    if 'GyroMag' not in df.columns:
        df['GyroMag'] = np.sqrt(df['GyroX']**2 + df['GyroY']**2 + df['GyroZ']**2)

    for start in range(0, len(df) - window_size, step_size):
        end = start + window_size
        window = df.iloc[start:end]

        # Incline-specific features: pitch angle and average vertical acceleration (AccZ)
        # Added a small epsilon to the denominator to avoid division by zero
        pitch_angle = np.arctan2(window['AccX'].mean(), np.sqrt(window['AccY'].mean()**2 + window['AccZ'].mean()**2) + 1e-9)
        avg_acc_z = window['AccZ'].mean()

        features = [
            window['AccX'].mean(), window['AccY'].mean(), window['AccZ'].mean(),
            window['GyroX'].mean(), window['GyroY'].mean(), window['GyroZ'].mean(),
            window['AccMag'].mean(), window['AccMag'].std(),
            window['GyroMag'].mean(), window['GyroMag'].std(),
            pitch_angle, # Feature 1: Tilt of the phone
            avg_acc_z    # Feature 2: Vertical acceleration component
        ]
        X.append(features)

    return np.array(X)

# --- Main Streamlit App Function ---
def main():
    st.title("🚶‍♂️ Activity & Incline Classifier")
    st.markdown("Upload your raw IMU data (accelerometer and gyroscope) to classify the activity and estimate caloric burn.")
    st.warning("⚠️ **Note:** The model expects a specific CSV format: headers are skipped, and columns must contain AccX, AccY, AccZ, GyroX, GyroY, GyroZ data in that order.")

    # --- User Inputs ---
    with st.sidebar:
        st.header("Parameters")

        # User uploads file
        uploaded_file = st.file_uploader("Choose a CSV file", type="csv")

        # User sets weight
        user_weight = st.number_input(
            "Your Weight (kg)",
            min_value=30.0,
            max_value=200.0,
            value=weight_kg,
            step=1.0
        )
        global weight_kg
        weight_kg = user_weight

    if uploaded_file is not None and model is not None:
        try:
            # 1. Load Data
            df = pd.read_csv(uploaded_file, skiprows=3)
            df.columns = ['AccX', 'AccY', 'AccZ', 'GyroX', 'GyroY', 'GyroZ']

            st.success("File uploaded and read successfully!")
            st.subheader("Raw Data Preview")
            st.dataframe(df.head())

            # 2. Preprocess and Feature Extraction
            X_new = extract_features_advanced(df)

            if len(X_new) == 0:
                st.error(f"Not enough data to extract features. Data points: {len(df)}. Minimum required for one window: {window_size}.")
                return

            # 3. Predict
            predictions = model.predict(X_new)

            # 4. Results Calculation

            # Count steps (windows) for each activity
            activity_counts = {
                'flat': np.sum(predictions == 'flat'),
                'incline': np.sum(predictions == 'incline'),
                'running': np.sum(predictions == 'running')
            }
            total_steps = sum(activity_counts.values())

            # Calculate durations
            duration_per_walking_window = step_size / sampling_rate # seconds
            total_recorded_duration_seconds = len(df) / sampling_rate
            total_processed_duration_seconds = len(X_new) * duration_per_walking_window

            # Calculate total walking time for each classified activity
            total_walking_seconds = {}
            total_calories_burned = 0

            for activity, count in activity_counts.items():
                seconds = count * duration_per_walking_window
                total_walking_seconds[activity] = seconds

                # Calorie burn for the classified windows
                calories = METs[activity] * weight_kg * (seconds / 3600) * 1.05
                total_calories_burned += calories

            # 5. Display Results
            st.header("📊 Classification Results")

            col1, col2, col3 = st.columns(3)

            col1.metric("Total Classified Steps (Windows)", f"{total_steps}")
            col2.metric("Approx. Distance Walked", f"{total_steps * step_length_m:.2f} meters")
            col3.metric("Estimated Total Calories Burned", f"🔥 {total_calories_burned:.2f} kcal")

            st.subheader("Activity Breakdown")

            # Display activity counts and time
            data_breakdown = {
                'Activity': list(activity_counts.keys()),
                'Classified Windows': list(activity_counts.values()),
                'Time Spent (minutes)': [s / 60 for s in total_walking_seconds.values()]
            }
            st.dataframe(pd.DataFrame(data_breakdown).set_index('Activity'))

            st.subheader("Duration Metrics")
            st.info(f"""
            - **Total Recorded Duration (Raw Data):** {total_recorded_duration_seconds / 60:.2f} minutes
            - **Total Processed Duration (Windows):** {total_processed_duration_seconds / 60:.2f} minutes
            - **Total Walking Time (Classified Windows):** {sum(total_walking_seconds.values()) / 60:.2f} minutes
            """)

        except Exception as e:
            st.error(f"An error occurred during processing: {e}")
            st.exception(e)
    elif model is None:
         st.markdown("Please fix the model loading error to continue.")
    else:
        st.info("Upload a CSV file to begin analysis.")

if __name__ == '__main__':
    main()

ModuleNotFoundError: No module named 'streamlit'